In [ ]:
!pip install CPLEX
from google.colab import drive
import pandas as pd
import numpy as np
import itertools
from itertools import product
import time

import cplex
from cplex.exceptions import CplexError
import sys
#np.set_printoptions()

drive.mount('/content/drive')

In [ ]:
def get_files(n, phi_frac, t_frac):
  df = pd.read_csv('/content/drive/MyDrive/Naumov_optimization/sorted_numbers.csv', delimiter=',', header=None)
  # read Time_probs.csv file
  df_probs = pd.read_csv('/content/drive/MyDrive/Naumov_optimization/Time_probs.csv', delimiter=';')
  # read B.csv file
  df_b = pd.read_csv('/content/drive/MyDrive/Naumov_optimization/new_new.csv')
  df_t = pd.read_csv('/content/drive/MyDrive/Naumov_optimization/T.csv', delimiter=';')

  X_0 = df_probs[['x0t1', 'x0t2', 'x0t3']].to_numpy()[:n]
  X_1 = df_probs[['x1t1', 'x1t2', 'x1t3']].to_numpy()[:n]

  b = df_b['b'].to_numpy()[:n]
  p = df_b['p'].to_numpy()[:n]

  p0 = 1 - p
  p1 = p

  max_b = np.sum(b[:n])
  phi = max_b * phi_frac

  timing_table = df_t.to_numpy()[:n, :]
  T_0 = timing_table[:, :3]
  T_1 = timing_table[:, 3:]
  t0_fast = T_0[:, 0]

  Tmax = np.sum(T_1[:, -1])
  T_m = Tmax * t_frac

  return df, df_probs, df_b, df_t, X_0, X_1, b, p, p0, p1, max_b, phi, timing_table, T_0, T_1, t0_fast, Tmax, T_m

df, df_probs, df_b, df_t, X_0, X_1, b, p, p0, p1, max_b, phi, timing_table, T_0, T_1, t0_fast, Tmax, T_m = get_files(3, 0.6, 0.8)

In [ ]:
# @title Функции для генерации
def generate_combinations(X, current_index, current_combination, combinations):
    if current_index == len(X):
        combinations.append(current_combination.copy())
        return
    for value in [2, 1, 0]:
        current_combination[current_index] = value
        generate_combinations(X, current_index + 1, current_combination, combinations)

def get_probX(X, p1):
  prob = 1
  print(X)
  print(p1)
  for i, x in enumerate(X):
    if x == 0: prob*=1-p1[i]
    else: prob*=p1[i]
  return prob

def generate_binary_combinations(n, current_index=0, current_combination=None, combinations=None):
    if combinations is None:
        combinations = []
    if current_combination is None:
        current_combination = [0] * n

    if current_index == n:
        combinations.append(current_combination.copy())
        return combinations

    for value in [0, 1]:
        current_combination[current_index] = value
        generate_binary_combinations(n, current_index + 1, current_combination, combinations)

    return combinations

def output_T(X, combinations):
    times_m = []
    times_probs = []
    for combination in combinations:
        times = []
        probs = []
        for i, value in enumerate(combination):
            if X[i] == 0:
                t = T_0[i][value]
                tp = X_0[i][value]
            else:
                t = T_1[i][value]
                tp = X_1[i][value]
            times.append(t)
            probs.append(tp)
        times_m.append(times)
        times_probs.append(np.prod(probs))
    return times_m, times_probs

def generate_arrays(tasks):
  # Задаем параметры
  n = tasks # Можно изменить для другого количества задач
  all_realizations = generate_binary_combinations(n)

  all_points = []
  all_times_var = []
  coeffs = []

  # Обработка каждой комбинации X
  for X in all_realizations:
      X = np.array(X)
      points_X = X * b[:n]  # b - предполагаемый массив коэффициентов
      all_points.append(points_X)
      prob_X = get_probX(X, p1[:n])  # Функция get_probX должна быть определена

      combinations = []
      generate_combinations(X, 0, [0] * len(X), combinations)
      times_m, times_probs = output_T(X, combinations)

      for tp in times_probs:
          coeffs.append(tp * prob_X)
      for tm in times_m:
          all_times_var.append(tm)
  return all_points, all_times_var, coeffs

In [ ]:
# Создаем экземпляр модели CPLEX
model = cplex.Cplex()
tasks = 1
constrs_half = 2**tasks * 3**tasks
#ограничений должно быть суммарно
constrs_half*2

# Задаем тип задачи - максимизация
model.objective.set_sense(model.objective.sense.maximize)

# Добавляем переменные x и y
# "B" означает бинарную переменную
variables = ['delta'+str(i) for i in range(constrs_half)] + ['u'+str(i) for i in range(tasks)]
model.variables.add(names=variables, types=[model.variables.type.binary]*(constrs_half+tasks))

In [ ]:
# @title OldAlg
'''
def output_T(X):
  times_m = []
  times_probs = []
  for a in [2, 1, 0]:
    for b in [2, 1, 0]:
      for c in [2, 1, 0]:
        #print(f'    {a, b, c}')
        if X[0] == 0:
          t1 = T_0[0][a]
          tp1 = X_0[0][a]
        else:
          t1 = T_1[0][a]
          tp1 = X_1[0][a]


        if X[1] == 0:
          t2 = T_0[1][b]
          tp2 = X_0[1][b]
        else:
          t2 = T_1[1][b]
          tp2 = X_1[1][b]


        if X[2] == 0:
          t3 = T_0[2][c]
          tp3 = X_0[2][c]
        else:
          t3 = T_1[2][c]
          tp3 = X_1[2][c]

        times_m.append([t1, t2, t3])
        #print('   ', times_m[-1])
        #print('   ', tp1, tp2, tp3)
        times_probs.append(tp1 * tp2 * tp3)
  return times_m, times_probs

def get_probX(X, p1):
  prob = 1
  for i, x in enumerate(X):
    if x == 0: prob*=1-p1[i]
    else: prob*=p1[i]
  return prob

all_realizations = []
all_points = []
all_times_var = []
coeffs = []
for x0 in [1, 0]:
  for x1 in [1, 0]:
    for x2 in [1, 0]:
      #print(x0, x1, x2, ' ', end = '')
      all_realizations.append([x0, x1, x2])
      X = np.array([x0, x1, x2])
      points_X = X * b
      all_points.append(points_X)
      #prob_X = np.prod(np.where(X == 1, p1, 1)) * np.prod(np.where(X == 0, p0, 1))
      prob_X = get_probX(X, p1)
      #print(prob_X)
      times_m, times_probs = output_T(X)
      for tp in times_probs:
        coeffs.append(tp*prob_X)
      for tm in times_m:
        all_times_var.append(tm)'''

In [ ]:
all_points, all_times_var, coeffs = generate_arrays(tasks)

In [ ]:
#x_i являются реализациями стратегии u. У нас суммарно 2 * D ограничений. D ограничений на время и D на баллы.
#первые D элементов в каждой строке, где только 1 единица указывают, какую из реализаций и для какого из временных исходов мы рассматриваем
#соответственно, у нас будет повторяться один и тот же паттерн для реализаций: 1024 = 2^10(для 10 задач)
#... будет повторяться 3^10 раз последовательно
#Аналогично для ограничений по T

all_points = np.array(all_points, dtype=float)
all_scorers_rep = np.repeat(all_points, 3**tasks, axis=0)
#all_scorers_rep = np.tile(all_points, (3**tasks, 1))

all_times_var = np.array(all_times_var, dtype=float)

In [ ]:
# Коэффициенты целевой функции
# для каждого элемента из delta нужна своя вероятность

model.objective.set_linear([('delta'+str(i), coeffs[i]) for i in range(constrs_half)] + [('u'+str(i), 0) for i in range(tasks)])

In [ ]:
all_scorers_rep_new = all_scorers_rep
all_times_var_new = all_times_var

model.linear_constraints.delete()

for v in range(constrs_half):
    # Создаем вектор коэффициентов для ограничения, где phi находится на диагонали
    constraint_vars = variables
    constraint_coefs = [0] * constrs_half  # начинаем с нулей для всех δ
    constraint_coefs[v] = phi  # заменяем φ на соответствующем месте
    constraint_coefs += list(-all_scorers_rep_new[v])  # добавляем коэффициенты для переменных u
    print(constraint_coefs)

    constraint_coefs = np.array(constraint_coefs, dtype='float')
    model.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=constraint_vars, val=constraint_coefs)],
        senses=['L'],
        rhs=[0]
    )

print()

for t in range(constrs_half):
    # Создаем вектор коэффициентов для ограничения, где -Tmax-T_m находится на диагонали
    constraint_vars = variables
    constraint_coefs1 = [0] * constrs_half  # начинаем с нулей для всех δ
    constraint_coefs1[t] = Tmax-T_m  # заменяем -T_m + Tmax на соответствующем месте
    constraint_coefs1 += list(all_times_var_new[t])  # добавляем коэффициенты для переменных u
    print(constraint_coefs1)

    constraint_coefs1 = np.array(constraint_coefs1, dtype='float')
    model.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=constraint_vars, val=constraint_coefs1)],
        senses=['L'],
        rhs=[float(Tmax)]
    )

In [ ]:
start_time = time.time()
model.solve()
end_time = time.time()

print(end_time - start_time)

# Вывод решения
print("Решение:")
#dot(deltaV, t)
deltas = []
for i in range(tasks):
  print(f"u{i+1} =", model.solution.get_values(f"u{i}"))

for i in range(constrs_half):
  print(f"delta{i+1} =", model.solution.get_values(f"delta{i}"))
  deltas.append(model.solution.get_values(f"delta{i}"))
deltas = np.array(deltas)
coeffs = np.array(coeffs)
print(np.dot(coeffs, deltas))